In [36]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold,GroupKFold,KFold,TimeSeriesSplit,GridSearchCV,train_test_split
from rfpimp import *
from tqdm import tqdm
from sklearn.metrics import f1_score,roc_auc_score,mean_squared_error,accuracy_score
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer,StandardScaler
from tqdm import tqdm
from datetime import date
from collections import Counter, defaultdict
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

pd.set_option('max.rows',8000)
pd.set_option('max.columns',30)

sns.set(rc={'figure.figsize':(13,10)})

# Data Loading

In [37]:
df = pd.read_csv("DDOS_vec.csv")

In [38]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,10.0.1.1-8.8.8.8-50659-53-17,10.0.1.1,50659,8.8.8.8,53,17,11/04/2022 11:47:45 PM,5004,4,0,156,0,39,39,39.0,...,0,0,0,0,3,47,0,0,0,0,0,0,0,0,Benign
1,10.0.1.1-8.8.8.8-43247-53-17,10.0.1.1,43247,8.8.8.8,53,17,11/04/2022 11:25:48 PM,5004,4,0,156,0,39,39,39.0,...,0,0,0,0,3,47,0,0,0,0,0,0,0,0,Benign
2,10.0.1.1-10.0.1.2-43048-55555-6,10.0.1.1,43048,10.0.1.2,55555,6,11/04/2022 11:26:09 PM,1,2,1,889,13,881,8,444.5,...,0,0,502,509,1,32,0,0,0,0,0,0,0,0,Benign
3,10.0.1.1-8.8.8.8-60936-53-17,10.0.1.1,60936,8.8.8.8,53,17,11/04/2022 11:25:28 PM,5004,4,0,156,0,39,39,39.0,...,0,0,0,0,3,47,0,0,0,0,0,0,0,0,Benign
4,10.0.1.1-10.0.1.2-43016-55555-6,10.0.1.1,43016,10.0.1.2,55555,6,11/04/2022 11:23:28 PM,1,2,1,841,13,833,8,420.5,...,0,0,502,509,1,32,0,0,0,0,0,0,0,0,Benign


In [39]:
# df.dtypes

# Some Data Preprocessing

In [40]:
#Removing columns with 1 unique value

temp = pd.DataFrame(df.nunique()).reset_index()
cols = temp[temp[0] != 1]['index'].to_list()

In [41]:
df_final = df[cols].copy()

Removing columns that can directly predict the test set

In [42]:
df_final.drop(['FWD Init Win Bytes','Total Length of Fwd Packet',
               'Fwd Packet Length Max','Fwd Packet Length Min',
              'Fwd Packet Length Mean','Packet Length Min','Packet Length Max',
              'Packet Length Mean','SYN Flag Count','Average Packet Size','Src Port','Flow IAT Max',
              'Flow IAT Mean','Bwd Packet Length Min','Fwd IAT Min','Fwd Packets/s',
               'Fwd Segment Size Avg','Fwd Seg Size Min','Flow Bytes/s',
              'Dst Port','Protocol',"Flow Duration","Total Fwd Packet","Total Bwd packets",
               "Flow IAT Min","Fwd IAT Mean","Fwd Header Length","Bwd Packets/s"],axis=1,inplace=True)

Correlatio basically tells us about the correlated the variables are to each other and to the target variable this function removes one randomly selected variable, out of two highly correlated ones. Don't check with target variable since it might remove that too. For checking with target variable remove "del dataset[colname]" statement and print the variables and manully drop them all.

In [43]:
def corrleation(dataset,threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix)):
        for j in range(i):
            if (corr_matrix.iloc[i,j]>=threshold) and (corr_matrix.columns[j] not in col_corr):
                print(corr_matrix.columns[i],corr_matrix.columns[j])
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
                if colname in dataset.columns:
                    print(colname)
                    del dataset[colname]

In [44]:
corrleation(df_final,0.9)

Fwd Packet Length Std Total Length of Bwd Packet
Fwd Packet Length Std
Bwd Packet Length Max Total Length of Bwd Packet
Bwd Packet Length Max
Bwd Packet Length Mean Total Length of Bwd Packet
Bwd Packet Length Mean
Flow Packets/s Total Length of Bwd Packet
Flow Packets/s
Fwd IAT Std Flow IAT Std
Fwd IAT Std
Fwd IAT Max Fwd IAT Total
Fwd IAT Max
Fwd PSH Flags Total Length of Bwd Packet
Fwd PSH Flags
Bwd Header Length Total Length of Bwd Packet
Bwd Header Length
Packet Length Std Total Length of Bwd Packet
Packet Length Std
Packet Length Variance Total Length of Bwd Packet
Packet Length Variance
PSH Flag Count Total Length of Bwd Packet
PSH Flag Count
Bwd Segment Size Avg Total Length of Bwd Packet
Bwd Segment Size Avg
Bwd Packet/Bulk Avg Flow IAT Std
Bwd Packet/Bulk Avg
Bwd Packet/Bulk Avg Bwd Bytes/Bulk Avg
Bwd Bulk Rate Avg Bwd Bytes/Bulk Avg
Bwd Bulk Rate Avg
Bwd Init Win Bytes Total Length of Bwd Packet
Bwd Init Win Bytes
Fwd Act Data Pkts Flow IAT Std
Fwd Act Data Pkts
Fwd Act Data

Distribution of data 

In [45]:
df_final['Label'].value_counts(normalize=True)

Malicious    0.683371
Benign       0.316629
Name: Label, dtype: float64

In [46]:
label_dic = {
    "Malicious":1,
    "Benign":0
}
df_final['Label'] = df_final['Label'].map(label_dic)

In [47]:
df_final.isnull().sum()

Flow ID                       0
Src IP                        0
Dst IP                        0
Timestamp                     0
Total Length of Bwd Packet    0
Flow IAT Std                  0
Fwd IAT Total                 0
ACK Flag Count                0
Bwd Bytes/Bulk Avg            0
Label                         0
dtype: int64

In [48]:
obj_cols = ['Flow ID','Src IP','Dst IP','Timestamp']

Some data statistics

In [49]:
df_final.groupby("Src IP").agg({"Label":['sum']})

,Label
,sum
Src IP,
10.0.1.1,0
10.0.11.1,2
10.0.11.2,298


In [50]:
df_final.groupby("Dst IP").agg({"Label":['sum']})

,Label
,sum
Dst IP,
10.0.1.2,0
10.0.11.1,298
239.255.255.250,2
8.8.8.8,0


In [51]:
to_drop = ['Flow ID','Src IP','Dst IP','Timestamp']
df_final.drop(to_drop,inplace=True,axis=1)

In [52]:
df_final

,Total Length of Bwd Packet,Flow IAT Std,Fwd IAT Total,ACK Flag Count,Bwd Bytes/Bulk Avg,Label
0,0,2889.060747,5004,0,156,0
1,0,2889.060747,5004,0,156,0
2,13,0.707107,1,3,0,0
3,0,2889.060747,5004,0,156,0
4,13,0.707107,1,3,0,0
5,0,2889.060747,5004,0,156,0
6,0,2889.060747,5004,0,156,0
7,0,2888.483397,5003,0,156,0
8,0,2889.060747,5004,0,156,0
9,0,2888.194765,5004,0,156,0


Columns had dataypes as float64 and int64 models like random forest don't like that, so below script finds the columns and then we convert them to float32 this might lose some information in the dataset.

In [53]:
#Finding those columns
float64_cols = list(df_final.select_dtypes(include='float64'))
int64_cols = list(df_final.select_dtypes(include='int64'))

In [54]:
#Conversion
df_final[float64_cols] = df_final[float64_cols].astype("float32")

df_final[int64_cols] = df_final[int64_cols].astype("int32")

In [55]:
df_final.isnull().sum()

Total Length of Bwd Packet    0
Flow IAT Std                  0
Fwd IAT Total                 0
ACK Flag Count                0
Bwd Bytes/Bulk Avg            0
Label                         0
dtype: int64

In [56]:
# To check infinite values since data set had huge values that are not suppported by 
# csv they turn to infinite we check these and change to nan
df_final.isin([np.inf, -np.inf]).sum()

Total Length of Bwd Packet    0
Flow IAT Std                  0
Fwd IAT Total                 0
ACK Flag Count                0
Bwd Bytes/Bulk Avg            0
Label                         0
dtype: int64

In [57]:
df_final.replace([np.inf, -np.inf], np.nan, inplace=True)

In [58]:
# filling those values with -99999
df_final.fillna(-99999,inplace=True)

Splitting to test and train

In [59]:
X_train,X_valid,y_train,y_valid = train_test_split(df_final.drop('Label',axis=1),df_final['Label'],stratify=df_final['Label'],random_state=22,test_size=0.2,shuffle=True)

The below method helps us to check if data is getting properly extraploated or not, now you only need to check if the roc is 0.50 between this hypothetical dataset that we create, if it is more than that that means there is a column in train that is leaking into test.

In [60]:
# X_train['is_valid'] = 0
# X_valid['is_valid'] = 1
# temp_df = X_train.append(X_valid)
# X_t_tr,X_t_val,y_t_tr,y_t_val = train_test_split(temp_df.drop('is_valid',axis=1),temp_df['is_valid'],stratify=temp_df['is_valid'],random_state=22,test_size=0.2)


In [61]:
# clf =RandomForestClassifier()
# clf.fit(X_t_tr.drop(to_drop,axis=1),y_t_tr)
# roc_auc_score(y_score=clf.predict_proba(X_t_val.drop(to_drop,axis=1))[:,0],y_true=y_t_val)

In [62]:
# to_drop = ['Fwd IAT Min','Fwd Packets/s','Fwd Segment Size Avg']

In [63]:
# imp = importances(clf,X_t_val.drop(to_drop,axis=1),y_t_val)
# imp

In [64]:
for i in X_train.columns:
    print(i)
    clf =RandomForestClassifier(random_state=22)
    clf.fit(X_train[i].values.reshape(-1,1),y_train)
    print(f1_score(y_pred=clf.predict(X_valid[i].values.reshape(-1,1)),y_true=y_valid))

Total Length of Bwd Packet
0.8275862068965517
Flow IAT Std
0.9836065573770492
Fwd IAT Total
1.0
ACK Flag Count
0.8275862068965517
Bwd Bytes/Bulk Avg
0.975609756097561


Use Stratifiedkfold for cross validation since the data is imbalanced , this helps generalize the model better

In [74]:
splits = 9
folds = StratifiedKFold(n_splits=splits, shuffle=True, random_state=22)
# final_preds = np.zeros((len(X_test), 1))
holdout_preds = np.zeros((len(X_valid),1))
oof_preds = []
final_preds_imp = pd.DataFrame()
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values, y_train.values)):
        print("Fold {}".format(fold_))
        X_trn,y_trn = X_train.iloc[trn_idx],y_train.iloc[trn_idx]
        X_val,y_val = X_train.iloc[val_idx],y_train.iloc[val_idx]
        clf  = RandomForestClassifier(random_state=22,n_jobs=-1)
        clf.fit(X_trn, y_trn)
        score = f1_score(y_pred=clf.predict(X_val),y_true=y_val)
        
#         print(score)
        oof_preds.append(score)
#         imp = importances(clf,X_val,y_val)
#         imp.reset_index(inplace=True)
#         final_preds_imp['Feature'] = X_val.columns
#         imp.rename(columns = {'Importance':f'importances{fold_}'},inplace=True)
#         final_preds_imp = pd.merge(final_preds_imp,imp,on='Feature')
        holdout_preds += clf.predict_proba(X_valid)[:,1].reshape(-1,1)
#         final_preds += clf.predict_proba(X_test)[:,1].reshape(-1,1)
# final_preds = final_preds/splits
holdout_preds = holdout_preds/splits
print(f"The Cross Validation score is :{sum(oof_preds)/splits}")

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
The Cross Validation score is :0.9937047126836249


Feature importance gives us the details about which features are important.

In [77]:
imp = importances(clf,X_trn,y_trn)

In [78]:
imp

,Importance
Feature,
Fwd IAT Total,0.157051
Flow IAT Std,0.006410
Total Length of Bwd Packet,0.000000
ACK Flag Count,0.000000
Bwd Bytes/Bulk Avg,0.000000
